<a href="https://colab.research.google.com/github/DolevSeren/DI-Bootcamp/blob/main/xpexcercise_W3D3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import pandas as pd

df = pd.read_csv("/content/sample_data/titanic.csv")

In [65]:
# בדיקה האם יש שורות כפולות
duplicates = df.duplicated()
print(f"Number of duplicate rows: {duplicates.sum()}")




Number of duplicate rows: 0


In [66]:
# סיכום כמות הערכים החסרים בכל עמודה
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])


Age         177
Cabin       687
Embarked      2
dtype: int64


In [67]:
median_age = df['Age'].median()
df['Age'] = df['Age'].fillna(median_age)
df['Cabin'] = df['Cabin'].fillna('Unknown')
most_frequent = df['Embarked'].mode()[0]
df['Embarked'] = df['Embarked'].fillna(most_frequent)


In [68]:
print(df.isnull().sum())


PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64


In [69]:
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1


In [70]:
df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


In [71]:
df = pd.get_dummies(df, columns=['Sex', 'Embarked', 'Title'], drop_first=True)


In [72]:
print(df.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'FamilySize', 'Sex_male', 'Embarked_Q',
       'Embarked_S', 'Title_Col', 'Title_Countess', 'Title_Don', 'Title_Dr',
       'Title_Jonkheer', 'Title_Lady', 'Title_Major', 'Title_Master',
       'Title_Miss', 'Title_Mlle', 'Title_Mme', 'Title_Mr', 'Title_Mrs',
       'Title_Ms', 'Title_Rev', 'Title_Sir'],
      dtype='object')


In [73]:
# גיל
Q1_age = df['Age'].quantile(0.25)
Q3_age = df['Age'].quantile(0.75)
IQR_age = Q3_age - Q1_age

# כרטיס
Q1_fare = df['Fare'].quantile(0.25)
Q3_fare = df['Fare'].quantile(0.75)
IQR_fare = Q3_fare - Q1_fare

# גבולות
lower_bound_age = Q1_age - 1.5 * IQR_age
upper_bound_age = Q3_age + 1.5 * IQR_age

lower_bound_fare = Q1_fare - 1.5 * IQR_fare
upper_bound_fare = Q3_fare + 1.5 * IQR_fare

# כמה חריגים יש
outliers_age = df[(df['Age'] < lower_bound_age) | (df['Age'] > upper_bound_age)]
outliers_fare = df[(df['Fare'] < lower_bound_fare) | (df['Fare'] > upper_bound_fare)]

print(f"Outliers in Age: {len(outliers_age)}")
print(f"Outliers in Fare: {len(outliers_fare)}")


Outliers in Age: 66
Outliers in Fare: 116


In [74]:
# גיל - קאפינג
df['Age'] = df['Age'].clip(lower=lower_bound_age, upper=upper_bound_age)

# מחיר - קאפינג
df['Fare'] = df['Fare'].clip(lower=lower_bound_fare, upper=upper_bound_fare)


In [75]:
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
print(numerical_cols)



Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare',
       'FamilySize'],
      dtype='object')


In [77]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[['Age', 'Fare']] = scaler.fit_transform(df[['Age', 'Fare']])



In [78]:
df['HasCabin'] = df['Cabin'].apply(lambda x: 0 if x == 'Unknown' else 1)


In [79]:
# מחזירים את Age המקורית לצורך בניית קבוצות
# נניח min_age=0.42 ו-max_age=80 (תוכל לשנות לפי הנתונים המקוריים שלך)
min_age = df['Age'].min()
max_age = df['Age'].max()
df['Age_original'] = df['Age'] * (max_age - min_age) + min_age


In [80]:
bins = [0, 12, 19, 35, 60, 100]
labels = ['Child', 'Teen', 'YoungAdult', 'Adult', 'Senior']
df['AgeGroup'] = pd.cut(df['Age_original'], bins=bins, labels=labels, right=False)


In [81]:
df = pd.get_dummies(df, columns=['AgeGroup'], drop_first=True)
